In [1]:
from ksoccer import KSoccer

In [11]:
processor = KSoccer(model='yolo11x.pt', source='video', path='inputs/equipo.mp4',ruta_carpeta = 'outputs/test4',save = True, scale_factor = 1,depurar=True)
processor.procesar()

Using device: cuda
File opened successfully.
Debugging enabled: True

0: 384x640 8 persons, 85.8ms
Speed: 0.0ms preprocess, 85.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.8ms
Speed: 2.1ms preprocess, 61.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Processing 8 player colors
DBSCAN labels: [-1  0  1  0  1  1  0  0]
Unique clusters: [-1  0  1]
Cluster sizes: [4, 3]
Main team colors updated - Team1: [     30.582      169.15       133.4], Team2: [     156.48      177.33      126.38]
Number of players in teams: [4, 3]
Frames - Team1: 0, Team2: 0, Total: 0
Possession - Team1: 0.0%, Team2: 0.0%
Imagen del frame guardada como frame_clasificado_1.png
Tiempo de procesamiento del frame 1: 1.5406 segundos

0: 384x640 9 persons, 75.0ms
Speed: 0.0ms preprocess, 75.0ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.6ms
Speed: 3.0ms preprocess, 66.6ms inference, 2.0ms post

In [9]:
processor = KSoccer(model='yolo11x.pt', source='image', path='inputs/image.png',ruta_carpeta = 'outputs/a',save = True, scale_factor = 1,depurar=True)
processor.procesar()

Using device: cuda
File opened successfully.
Debugging enabled: True

0: 480x640 (no detections), 77.1ms
Speed: 3.3ms preprocess, 77.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 sports ball, 86.3ms
Speed: 1.0ms preprocess, 86.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Imagen del frame guardada como frame_clasificado_0.png


In [13]:
from video_converter import crear_video_desde_imagenes
ruta_carpeta = 'outputs/test4'  # Cambia esto por tu ruta
nombre_video_salida = 'output_videos/x.mp4'  # Nombre del video de salida
crear_video_desde_imagenes(ruta_carpeta, nombre_video_salida)

Video creado correctamente: output_videos/x.mp4
